In [1]:
import numpy as np

# Generate Data

In [33]:
Omega = np.array([[0,1],[0,1]])
observation_resolution = 5
observations = (np.array(np.meshgrid(
                    *(
                        np.linspace(bound[0], bound[1], observation_resolution)
                        for bound in Omega
                    ))
            ).reshape(len(Omega), -1).T)
std_factor = 0.5
k = lambda x: np.array([np.exp(-np.linalg.norm(point-x)**2/std_factor)/np.sqrt(std_factor*np.pi)**Omega.shape[0] for point in observations])
true_sources = np.array([[0.1,0.3], [0.25,0.8], [0.7,0.5]])
true_weights = np.array([5,-3,7])
target = sum([weight*k(source) for weight, source in zip(true_weights, true_sources)])
norm_K_star = np.sqrt(len(observations)/(std_factor*np.pi))
norm_K_star_L = 2*np.sqrt(len(observations)*Omega.shape[0]/(np.pi*std_factor**3))/std_factor

In [34]:
def p_raw(u, k, target):
    def p_u(x):
        Ku = sum(k(x)*c for c, x in zip(u.coefficients, u.support))
        inner = Ku-target
        return -np.dot(inner, k(x))
    return p_u

p = lambda u: p_raw(u, k=k, target=target)

In [35]:
def grad_P_raw(k, target, observations, std_factor):
    def grad_P(u, x):
        def p_u(x):
            Ku = sum(k(x)*c for c, x in zip(u.coefficients, u.support))
            inner = Ku-target
            return -np.dot(inner, k(x))
        p = p_u(u)
        Ku = sum(k(x)*c for c, x in zip(u.coefficients, u.support))
        inner = Ku-target
        second_inner = k(x)*2/std_factor
        vector_part = sum([inr*second_inr*(obs-x) for inr, second_inr, obs in zip(inner, second_inner, observations)])
        return -np.sign(p(x))*vector_part
    return grad_P

grad_P = grad_P_raw(k, target, observations, std_factor)

In [30]:
target

array([3.15191494, 3.81351948, 3.95712709, 3.50943456, 2.61142507,
       3.66011727, 4.6079549 , 5.00984175, 4.64102819, 3.57307225,
       3.14489828, 4.18610067, 4.83961186, 4.72874385, 3.78584505,
       1.87996289, 2.75232252, 3.49495837, 3.67245274, 3.08807258,
       0.62956544, 1.18183781, 1.80673047, 2.13326966, 1.92170735])

In [22]:
observations

array([[0.  , 0.  ],
       [0.25, 0.  ],
       [0.5 , 0.  ],
       [0.75, 0.  ],
       [1.  , 0.  ],
       [0.  , 0.25],
       [0.25, 0.25],
       [0.5 , 0.25],
       [0.75, 0.25],
       [1.  , 0.25],
       [0.  , 0.5 ],
       [0.25, 0.5 ],
       [0.5 , 0.5 ],
       [0.75, 0.5 ],
       [1.  , 0.5 ],
       [0.  , 0.75],
       [0.25, 0.75],
       [0.5 , 0.75],
       [0.75, 0.75],
       [1.  , 0.75],
       [0.  , 1.  ],
       [0.25, 1.  ],
       [0.5 , 1.  ],
       [0.75, 1.  ],
       [1.  , 1.  ]])